###### As home range of H5 AIV in each month, we combined the virus diffusion in all values of HPD in each month. Corresponding utilization distribution of the virus was the virus diffusion in each value of HPD in every month. For each month slice across a year, we sampled 1000 points at random geographical coordinates within the home range of viruses. 

In [22]:
import json
import pandas as pd
import os
import shutil
import random
import numpy as np
#from geojson_utils import point_in_multipolygon
from shapely import geometry as gt
from shapely import ops
from shapely.prepared import prep
from shapely.geometry import MultiPolygon
from shapely.geometry import Point

In [23]:
## load a land polygon to test the random points were in land
land = r"../data/world.json"
f1 = open(land)
WORLD_LAND = prep(gt.shape(json.load(f1)['features'][0]['geometry']))
f1.close()

In [24]:
def json2df(file):
    # convert the highest posterior density (HPD) of the inferred virus to dataframe
    f = open(file)
    test = json.load(f)
    f.close()
    df = pd.DataFrame(test['layers'][0]['areas'])
    df.attributes = df['attributes'].apply(lambda x:float(x['HPD']))
    df['corrdinates'] = df['polygon'].apply(lambda cord:
                                                            [list(map(lambda x:
                                                                      [x['xCoordinate'],x['yCoordinate']],cord['coordinates']
                                                                     )
                                                                 )
                                                            ])
    df.drop('polygon',axis = 1,inplace = True)
    df['polygon']=df.corrdinates.apply(lambda x:gt.shape({"type":"MultiPolygon","coordinates":[x]}))
    #groups = df.groupby(['startTime','attributes'])
    #out_df = pd.DataFrame(groups.apply(lambda x:ops.unary_union(list(x.polygon))),columns=['polygon'])
    out_df = pd.DataFrame(ops.unary_union(list(df.polygon)),columns=['polygon'])
    multi1 = MultiPolygon(ops.unary_union(list(df.polygon)))
    return multi1

In [25]:
def json2df1(file):
    #file = virus_file1
    f = open(file)
    test = json.load(f)
    f.close()
    df = pd.DataFrame(test['layers'][0]['areas'])
    df.attributes = df['attributes'].apply(lambda x:float(x['HPD']))
    df['corrdinates'] = df['polygon'].apply(lambda cord:
                                                                [list(map(lambda x:
                                                                          [x['xCoordinate'],x['yCoordinate']],cord['coordinates']
                                                                         )
                                                                     )
                                                                ])
    df.drop('polygon',axis = 1,inplace = True)
    df['polygon']=df.corrdinates.apply(lambda x:gt.shape({"type":"MultiPolygon","coordinates":[x]}))
    groups = df.groupby(['startTime','attributes'])
    out_df = pd.DataFrame(groups.apply(lambda x:ops.unary_union(list(x.polygon))),columns=['polygon'])
    out_df = out_df.reset_index(level='attributes')
    return out_df

In [26]:
def get_random(convex_hull,num):
    '''
    convex_hull:  home range of the Mulitpolygon in one time(month)
    out: points within conver_hull and land
    '''
    #print(union_polygon)
    #num = 100
    polygon_shp = convex_hull
    #prep_shp = prep(polygon_shp)
    #stime = str(convex_hull.name)
    minx, miny, maxx, maxy = polygon_shp.bounds
    
    tlist_of_points = []
    counter = 0
    while counter < num:
        pnt = gt.Point(random.uniform(minx, maxx), random.uniform(miny, maxy))
        if polygon_shp.contains(pnt) and WORLD_LAND.contains(pnt):
            #tlist_of_points.append([stime,pnt,pnt.coords[:][0][1],pnt.coords[:][0][0],int(stime.split('/')[1])])
            tlist_of_points.append([pnt,pnt.coords[:][0][1],pnt.coords[:][0][0]])
            counter += 1 
    out = pd.DataFrame(tlist_of_points,columns=['point','lat','lon'])
    return out

In [27]:
def inclusion_relation(point,polygons):
    '''
    # point - one random point in one time
    # polygon - virus polygons, polygons.loc[point.startTime] get the polygon in this time
    # out - the spatial relationship between point and these polygons ,and are randed by 0.1-0.95HPD
    '''
    out = list(map(lambda x:x.contains(point.point),polygons.loc[point.startTime]['polygon']))
    return out

In [28]:
'''def stream(virus_path,num,out_path):

    df_merge = json2df(virus_path)
    out_df = json2df1(virus_path)
    #df_v = json2df1(virus_path)
    #df_v = df_v.reset_index(level='attributes')

    # get the convex_hull of every timesilces
    #time_group = df_v.groupby('startTime')
    #time_hull = pd.DataFrame(time_group.apply(lambda x:ops.unary_union(list(x.polygon)).convex_hull),columns = ['convex_hull'])

    random_num = num
    random_df = pd.DataFrame(get_random(df_merge,num)).reset_index(drop=True)
    
    # The spatial relationship between random points and each HPD 
    #spatial_relate = random_df.apply(inclusion_relation,axis=1,**{'polygons':df_v})
    #hpd_l = list(np.arange(10,100,5)/100)
    #spatial_relate = pd.DataFrame(list(map(lambda x:x, spatial_relate)),columns=hpd_l)
    #random_relate = pd.concat([random_df,spatial_relate],axis=1).set_index('startTime')
    
    #random_df.to_csv(out_path,index=False)
    
    listtt = []
    for i in set(out_df.index):
        random_df['startTime'] = i
        listtt.append(random_df)
    full_random = pd.concat(listtt).reset_index(drop=True)
    
    
    spatial_relate = full_random.apply(inclusion_relation,axis=1,**{'polygons':out_df})
    hpd_l = list(np.arange(10,100,5)/100)
    spatial_relate = pd.DataFrame(list(map(lambda x:x,spatial_relate)),columns=hpd_l)
    random_relate = pd.concat([full_random,spatial_relate],axis=1).set_index('startTime')
    random_relate.insert(3,'month',pd.to_datetime(random_relate.index).month)
    random_relate.to_csv(out_path)'''

"def stream(virus_path,num,out_path):\n\n    df_merge = json2df(virus_path)\n    out_df = json2df1(virus_path)\n    #df_v = json2df1(virus_path)\n    #df_v = df_v.reset_index(level='attributes')\n\n    # get the convex_hull of every timesilces\n    #time_group = df_v.groupby('startTime')\n    #time_hull = pd.DataFrame(time_group.apply(lambda x:ops.unary_union(list(x.polygon)).convex_hull),columns = ['convex_hull'])\n\n    random_num = num\n    random_df = pd.DataFrame(get_random(df_merge,num)).reset_index(drop=True)\n    \n    # The spatial relationship between random points and each HPD \n    #spatial_relate = random_df.apply(inclusion_relation,axis=1,**{'polygons':df_v})\n    #hpd_l = list(np.arange(10,100,5)/100)\n    #spatial_relate = pd.DataFrame(list(map(lambda x:x, spatial_relate)),columns=hpd_l)\n    #random_relate = pd.concat([random_df,spatial_relate],axis=1).set_index('startTime')\n    \n    #random_df.to_csv(out_path,index=False)\n    \n    listtt = []\n    for i in set(out

In [34]:
def stream(virus_path,num,out_path):

    df_merge = json2df(virus_path)
    out_df = json2df1(virus_path)
    #df_v = json2df1(virus_path)
    #df_v = df_v.reset_index(level='attributes')

    # get the convex_hull of every timesilces
    #time_group = df_v.groupby('startTime')
    #time_hull = pd.DataFrame(time_group.apply(lambda x:ops.unary_union(list(x.polygon)).convex_hull),columns = ['convex_hull'])

    random_num = num
    random_df = pd.DataFrame(get_random(df_merge,num)).reset_index(drop=True)
    
    # The spatial relationship between random points and each HPD 
    #spatial_relate = random_df.apply(inclusion_relation,axis=1,**{'polygons':df_v})
    #hpd_l = list(np.arange(10,100,5)/100)
    #spatial_relate = pd.DataFrame(list(map(lambda x:x, spatial_relate)),columns=hpd_l)
    #random_relate = pd.concat([random_df,spatial_relate],axis=1).set_index('startTime')
    
    #random_df.to_csv(out_path,index=False)
    
    listtt = []
    for i in set(out_df.index):
        random_df['startTime'] = i
        temp = random_df.copy()
        listtt.append(temp)

    full_random = pd.concat(listtt).reset_index(drop=True)
    #return full_random
    
    spatial_relate = full_random.apply(inclusion_relation,axis=1,**{'polygons':out_df})
    hpd_l = list(np.arange(10,100,5)/100)
    spatial_relate = pd.DataFrame(list(map(lambda x:x,spatial_relate)),columns=hpd_l)
    random_relate = pd.concat([full_random,spatial_relate],axis=1).set_index('startTime')
    random_relate.insert(3,'month',pd.to_datetime(random_relate.index).month)
    random_relate.to_csv(out_path)

In [36]:
virus_file1 = r'../data/HPD_10_95_2.3.2.1.json'
outfile1 = r'../data/random_df_2.3.2.1.csv'
random_num1 = 1000
stream(virus_path=virus_file1,num=random_num1,out_path=outfile1)

/Users/qiqiy/anaconda3/envs/jupyterlab-debugger/lib/python3.10/site-packages/pandas/core/frame.py:710: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  data = list(data)
/Users/qiqiy/anaconda3/envs/jupyterlab-debugger/lib/python3.10/site-packages/pandas/core/frame.py:710: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  data = list(data)
/Users/qiqiy/anaconda3/envs/jupyterlab-debugger/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:1981: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  result[:] = value

In [38]:
virus_file2 = r'../data/HPD_10_95_2.3.4.4.json'
outfile2 = r'../data/random_df_2.3.4.4.csv'
random_num2 = 1000
stream(virus_path=virus_file2,num=random_num2,out_path=outfile2)

/Users/qiqiy/anaconda3/envs/jupyterlab-debugger/lib/python3.10/site-packages/pandas/core/frame.py:710: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  data = list(data)
/Users/qiqiy/anaconda3/envs/jupyterlab-debugger/lib/python3.10/site-packages/pandas/core/frame.py:710: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  data = list(data)
/Users/qiqiy/anaconda3/envs/jupyterlab-debugger/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:1981: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  result[:] = value

### DRAFT

In [72]:
temp = pd.read_csv(r'../data/test.csv',index_col=0)

In [110]:
temp.point = temp.apply(lambda x:Point(x.lon,x.lat),axis=1)

In [112]:
file = virus_file1
f = open(file)
test = json.load(f)
f.close()
df = pd.DataFrame(test['layers'][0]['areas'])
df.attributes = df['attributes'].apply(lambda x:float(x['HPD']))
df['corrdinates'] = df['polygon'].apply(lambda cord:
                                                            [list(map(lambda x:
                                                                      [x['xCoordinate'],x['yCoordinate']],cord['coordinates']
                                                                     )
                                                                 )
                                                            ])
df.drop('polygon',axis = 1,inplace = True)
df['polygon']=df.corrdinates.apply(lambda x:gt.shape({"type":"MultiPolygon","coordinates":[x]}))
groups = df.groupby(['startTime','attributes'])
out_df = pd.DataFrame(groups.apply(lambda x:ops.unary_union(list(x.polygon))),columns=['polygon'])
out_df = out_df.reset_index(level='attributes')

In [113]:
listtt = []
for i in set(out_df.index):
    temp['startTime'] = i
    listtt.append(temp)
full_random = pd.concat(listtt)

In [132]:
full_random = full_random.reset_index(drop=True)

In [147]:
spatial_relate = full_random.head(20).apply(inclusion_relation,axis=1,**{'polygons':out_df})
hpd_l = list(np.arange(10,100,5)/100)
spatial_relate = pd.DataFrame(list(map(lambda x:x,spatial_relate)),columns=hpd_l)
random_relate = pd.concat([full_random.head(20),spatial_relate],axis=1).set_index('startTime')

In [148]:
random_relate.insert(3,'month',pd.to_datetime(random_relate.index).month)

In [149]:
random_relate

point        lat  \
startTime                                                            
2014/09/15  POINT (117.1005624595494 28.59784372324516)  28.597844   
2014/09/15  POINT (41.40000227152473 40.39816307069079)  40.398163   
2014/09/15  POINT (56.86316264172859 40.25740622884268)  40.257406   
2014/09/15   POINT (118.767826540262 33.72302106159697)  33.723021   
2014/09/15  POINT (41.94750510638828 28.05866835889827)  28.058668   
2014/09/15  POINT (71.66157568766612 29.98494564480833)  29.984946   
2014/09/15  POINT (5.352389817725181 12.24066824758956)  12.240668   
2014/09/15  POINT (108.0614432997161 34.61275670046543)  34.612757   
2014/09/15  POINT (41.42738387366758 43.28584151336398)  43.285842   
2014/09/15  POINT (39.17722250498873 32.39749211994942)  32.397492   
2014/09/15  POINT (117.1005624595494 28.59784372324516)  28.597844   
2014/09/15  POINT (41.40000227152473 40.39816307069079)  40.398163   
2014/09/15  POINT (56.86316264172859 40.25740622884268)  40.257406   
2014/09/15   POINT (118.767826540262 33.72302106159697)  33.723021   
2014/09/15  POINT (41.94750510638828 28.05866835889827)  28.058668   
2014/09/15  POINT (71.66157568766612 29.98494564480833)  29.984946   
2014/09/15  POINT (5.352389817725181 12.24066824758956)  12.240668   
2014/09/15  POINT (108.0614432997161 34.61275670046543)  34.612757   
2014/09/15  POINT (41.42738387366758 43.28584151336398)  43.285842   
2014/09/15  POINT (39.17722250498873 32.39749211994942)  32.397492   

                   lon  month    0.1   0.15    0.2   0.25    0.3   0.35  ...  \
startTime                                                                ...   
2014/09/15  117.100562      9  False  False  False  False  False  False  ...   
2014/09/15   41.400002      9  False  False  False  False  False  False  ...   
2014/09/15   56.863163      9  False  False  False  False  False  False  ...   
2014/09/15  118.767827      9  False  False  False  False  False  False  ...   
2014/09/15   41.947505      9  False  False  False  False  False  False  ...   
2014/09/15   71.661576      9  False  False  False  False  False  False  ...   
2014/09/15    5.352390      9  False  False  False  False  False  False  ...   
2014/09/15  108.061443      9  False  False  False  False  False  False  ...   
2014/09/15   41.427384      9  False  False  False  False  False  False  ...   
2014/09/15   39.177223      9  False  False  False  False  False  False  ...   
2014/09/15  117.100562      9  False  False  False  False  False  False  ...   
2014/09/15   41.400002      9  False  False  False  False  False  False  ...   
2014/09/15   56.863163      9  False  False  False  False  False  False  ...   
2014/09/15  118.767827      9  False  False  False  False  False  False  ...   
2014/09/15   41.947505      9  False  False  False  False  False  False  ...   
2014/09/15   71.661576      9  False  False  False  False  False  False  ...   
2014/09/15    5.352390      9  False  False  False  False  False  False  ...   
2014/09/15  108.061443      9  False  False  False  False  False  False  ...   
2014/09/15   41.427384      9  False  False  False  False  False  False  ...   
2014/09/15   39.177223      9  False  False  False  False  False  False  ...   

              0.5   0.55    0.6   0.65    0.7   0.75    0.8   0.85    0.9  \
startTime                                                                   
2014/09/15  False   True   True   True   True   True   True   True   True   
2014/09/15  False  False  False  False  False  False  False  False  False   
2014/09/15  False  False  False  False  False  False  False  False  False   
2014/09/15  False  False   True   True   True   True   True   True   True   
2014/09/15  False  False  False  False  False  False  False  False  False   
2014/09/15  False  False  False  False  False  False  False  False  False   
2014/09/15  False  False  False  False  False  False  False  False  False   
2014/09/15  False  False  False  False  False  False  F